In [1]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import flask
from flask_cors import CORS
import os
import numpy as np
import pickle
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import entropy
import networkx as nx

In [2]:
app = dash.Dash('drug-discovery')
server = app.server

In [3]:
df = pd.read_feather('./df_final.feather')
with open('./dash-drug-discovery-demo/thresh_to_graph.pkl','rb') as f:
    thresh_to_graph = pickle.load(f)
with open('./dash-drug-discovery-demo/thresh_to_XnYn.pkl','rb') as f:
    thresh_to_XnYn = pickle.load(f)
with open('./dash-drug-discovery-demo/thresh_to_pos.pkl','rb') as f:
    thresh_to_pos = pickle.load(f)

In [4]:
df.head()

,PAGE,IMG_URL,NAME,is_strip,stars,Topic1,Topic2,Topic3,Topic4,Topic5,...,Topic16,Topic17,Topic18,temp_id,topic1_for_disp,topic2_for_disp,topic3_for_disp,threshold,Xn,Yn
0,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/structures/DB01000/thu...,Jaburritos,True,4.5 stars,0.031864,0.001086,0.005289,0.007907,0.006254,...,0.054163,0.013609,0.022229,0,"Cost & Quality: food, good, price","Pizza: slice, wing, pizza","Lunch: lunch, location, place",0.55,0.092914,-0.110497
1,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/structures/DB01001/thu...,Le Thai,False,4.0 stars,0.000827,0.081257,0.002911,0.053077,0.575448,...,0.173229,0.003961,0.005508,1,"Italian: salad, pasta, bread","Wait Time: minute, table, wait","Fine Dining: restaurant, dinner, dessert",0.55,-0.701477,0.636569
2,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/structures/DB01002/thu...,Sugar Factory,True,3.5 stars,0.007088,0.222558,0.009247,0.107580,0.003468,...,0.318069,0.021889,0.019387,2,"Bars: drink, beer, bar","Fine Dining: restaurant, dinner, dessert","Italian: salad, pasta, bread",0.55,NaN,NaN
3,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/structures/DB01003/thu...,Rollin' Smoke BBQ,False,4.0 stars,0.004860,0.000439,0.021188,0.011603,0.042555,...,0.033540,0.616419,0.107080,3,"High Customer Satisfaction: food, service, great","Asian: noodle, thai, rice","Fine Dining: restaurant, dinner, dessert",0.55,-0.347040,0.157727
4,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/structures/DB01004/thu...,PublicUs,False,4.5 stars,0.008916,0.428269,0.010909,0.016861,0.001839,...,0.101793,0.177481,0.037548,4,"High Customer Satisfaction: food, service, great","Cost & Quality: food, good, price","Italian: salad, pasta, bread",0.55,NaN,NaN


In [5]:
# dont worry about this cell
if 'DYNO' in os.environ:
    app.scripts.append_script({
        'external_url': 'https://cdn.rawgit.com/chriddyp/ca0d8f02a1659981a0ea7f013a378bbd/raw/e79f3f789517deec58f41251f7dbb6bee72c44ab/plotly_ga.js'
    })
    
BACKGROUND = 'rgb(230, 230, 230)'

COLORSCALE = [ [0, "rgb(244,236,21)"], [0.3, "rgb(249,210,41)"], [0.4, "rgb(134,191,118)"],
                [0.5, "rgb(37,180,167)"], [0.65, "rgb(17,123,215)"], [1, "rgb(54,50,153)"] ]

In [6]:
# arbitrary threshold for deciding whether 2 observations are 'similar' or not
threshold_all = [0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62]
def th_mark(x):
    if x==np.min(threshold_all):
        return 'Low'
    elif x==np.max(threshold_all):
        return 'High'
    else:
        return ''
    
threshold_mark = {str(th):th_mark(th) for th in threshold_all}

In [7]:
def update_slider_mark(slider_mark, font_size):
    # update display style of position markers for the slider
    slider_mark_updated = {}
    for position in slider_mark:
        slider_mark_updated[position] = {
            'label': slider_mark[position],
            'style': {'fontSize':font_size, 'font-family': 'Arial'}
        }
    return slider_mark_updated

threshold_mark_updated = update_slider_mark(threshold_mark, 15)

In [8]:
# figure data is the data object we pass into figure function 
# molecules will be the selected business
# change this function for our needs

def add_markers(selected_threshold, df, molecules, plot_type = 'scatter' ):
    indices = []
    drug_data = df
    for m in molecules:
        # this is the text attribute of data object 
        hover_text = drug_data.NAME.tolist()
        for i in range(len(hover_text)):
            if m == hover_text[i]:
                indices.append(i)

    trace_markers = []
    for point_number in indices:
        trace = dict(
            x = [drug_data.loc[point_number,'Xn']],
            y = [drug_data.loc[point_number,'Yn']],
            marker = dict(
                color = 'black',
                size = 10,
                opacity = 0.6,
                symbol = 'cross'
            ),
            type = plot_type
        )
        trace_markers.append(trace)
    print(trace_markers)   
    Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip = thresh_to_XnYn[selected_threshold]

    # define a trace for plotly
    trace_nodes1 = dict(type='scatter', 
                        x=Xn_strip, 
                        y=Yn_strip,
                        mode='markers',
                        marker=dict(symbol='dot', 
                                    size=10, color='rgb(255,0,0)'),
                        name='On The Strip',
                        showlegend=True, 
                        text = [],
                        hoverinfo='text',
                        visible=True)
    trace_nodes2 = dict(type='scatter', 
                        x=Xn_notstrip, 
                        y=Yn_notstrip,
                        mode='markers',
                        marker=dict(symbol='dot', 
                                    size=10, color='rgb(0, 0, 255)'),
                        name='Not on The Strip',
                        showlegend=True, 
                        text = [],
                        hoverinfo='text',
                        visible=True)        
    
    # record the coordinates of the ends of edges
    Xe = []
    Ye = []
    G = thresh_to_graph[selected_threshold]
    for e in G.edges():
        pos = thresh_to_pos[selected_threshold]
        Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
        Ye.extend([pos[e[0]][1], pos[e[1]][1], None])

    # trace_edges defines the graph edges as a trace of type scatter (line)
    trace_edges=dict(type='scatter',
                     mode='lines',
                     x=Xe,
                     y=Ye,
                     line=dict(width=0.1, color='rgb(51, 51, 51)'),
                     hoverinfo='none', showlegend=False)
    traces = [trace_nodes1] + [trace_nodes2] + [trace_edges]+trace_markers
    return traces

# change this function 
def scatter_plot_3d(selected_threshold, xlabel='LogP', ylabel='pkA', plot_type='scatter', markers=[]):
    df_for_plot = df.copy()
    df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
    df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
    x= df_for_plot['Xn'] 
    y= df_for_plot['Yn']
#     size= df_for_plot['MW']
#     color= df_for_plot['MW']
    def axis_template_2d(title):
        return dict(
            xgap = 10, ygap = 10,
            backgroundcolor = BACKGROUND,
            gridcolor = 'rgb(255, 255, 255)',
            title = title,
            zerolinecolor = 'rgb(255, 255, 255)',
            color = '#444'
        )
    # change data
    data = [dict(
        x = x,
        y = y,
        mode = 'markers',
        text = df_for_plot['NAME'],
        type = plot_type,
    ) ]
    # change layour
    layout = dict(
        font = dict( family = 'Raleway' ),
        hovermode = 'hoverclosest',
        margin = dict( r=20, t=0, l=0, b=0 ),
        showlegend = False
    )
    # change scatter
    if plot_type in ['scatter']:
        layout['xaxis'] = axis_template_2d(xlabel)
        layout['yaxis'] = axis_template_2d(ylabel)
        layout['plot_bgcolor'] = BACKGROUND
        layout['paper_bgcolor'] = BACKGROUND
        
    # keep this
    
    data = data + add_markers(selected_threshold, df_for_plot, markers, plot_type = plot_type )

    return dict(data=data, layout=layout)

selected_threshold = threshold_all[0]
FIGURE = scatter_plot_3d(selected_threshold)
df_for_plot = df.copy()
df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
STARTING_DRUG = df_for_plot.loc[0,'NAME']
# DRUG_DESCRIPTION = df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['DESC'].iloc[0]
STAR_RATING =df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['stars'].iloc[0]
DRUG_IMG = df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['IMG_URL'].iloc[0]

[]


In [ ]:
app.layout = html.Div([
    # Row 1: Header and Intro text
    html.Div([
        html.Div([
            html.Div([
                html.P('HOVER over a drug in the graph to the right to see its structure to the left.'),
                html.P('SELECT a drug in the dropdown to add it to the drug candidates at the bottom.')
            ], style={'margin-left': '10px'}),
            dcc.Dropdown(id='chem_dropdown',
                        multi=True,
                        value=[ STARTING_DRUG ],
                        options=[{'label': i, 'value': i} for i in df_for_plot['NAME'].tolist()]),
            ], className='twelve columns' )

    ], className='row' ),

    # Row 2: Hover Panel and Graph
    html.Div([
        html.Div([

            html.Img(id='chem_img', src=DRUG_IMG ),

            html.Br(),

             ### Restaurant name           
            html.P('Restaurant name',
                  id='rest_name',
                  style=dict( maxHeight='400px', fontSize='12px')),
            
            html.A(STARTING_DRUG,
                  id='chem_name',
                  href="https://www.drugbank.ca/drugs/DB01002",
                  target="_blank"),

            html.Br(),
### Star rating            
            html.P('Average Rating',
                  id='star_rating',
                  style=dict( maxHeight='400px', fontSize='12px' )),

            html.P(STAR_RATING,
                  id='star_rating',
                  style=dict( maxHeight='400px', fontSize='12px' )),
            
### Words in a topic
            html.P('Major topics in reviews',
                  id='review_topics',
                  style=dict( maxHeight='400px', fontSize='12px' )),            

        ], className='three columns', style=dict(height='300px')),

        html.Div([
            dcc.Graph(id='clickable-graph',
                      style=dict(width='700px'),
                      hoverData=dict(points=[dict(pointNumber=0)] ),
                      figure=FIGURE ),

        ], className='nine columns', style=dict(textAlign='center')),
    html.Div([
    html.H2('Similarity Cutoff'),
    dcc.Slider(
        id='threshold-slider',
        min=min(threshold_all),
        max=max(threshold_all),
        value=selected_threshold,
        step=None,
        marks=threshold_mark_updated
    ),
    ], style={'width': '47%','marginBottom': 0, 'marginTop': 0, 'marginLeft':'auto', 'marginRight':'auto',
              'fontSize':12, 'font-family': 'Arial'}
    )
    ], className='row' ),

], className='container')


@app.callback(
    Output('clickable-graph', 'figure'),
    [Input('chem_dropdown', 'value'), Input('threshold-slider', 'value')])
def highlight_molecule(chem_dropdown_values, selected_threshold):
    return scatter_plot_3d(selected_threshold=selected_threshold, 
                           markers = chem_dropdown_values, plot_type = 'scatter')



def dfRowFromHover(hoverData,selected_threshold):
    ''' Returns row for hover point as a Pandas Series '''
    if hoverData is not None:
        if 'points' in hoverData:
            firstPoint = hoverData['points'][0]
            if 'pointNumber' in firstPoint:
                point_number = firstPoint['pointNumber']
#                 molecule_name = str(FIGURE['data'][0]['text'][point_number]).strip()

                df_for_plot = df.copy()
                df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
                df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
                molecule_name = df_for_plot.loc[point_number,'NAME']
                return df_for_plot.loc[df_for_plot['NAME'] == molecule_name]
    return pd.Series()


@app.callback(
    dash.dependencies.Output('chem_dropdown', 'options'),
    [dash.dependencies.Input('threshold-slider', 'value')])
def set_dropdown_options(selected_threshold):
    df_for_plot = df.copy()
    df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
    df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
    return [{'label': i, 'value': i} for i in df_for_plot['NAME'].tolist()]

@app.callback(
    dash.dependencies.Output('chem_dropdown', 'value'),
    [dash.dependencies.Input('chem_dropdown', 'options')])
def set_dropdown_value(available_options):
    return available_options[0]['value']

@app.callback(
    Output('chem_name', 'children'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def return_molecule_name(hoverData,selected_threshold):
    if hoverData is not None:
#         print(hoverData)
        if 'points' in hoverData:
            firstPoint = hoverData['points'][0]
            if 'pointNumber' in firstPoint:
                point_number = firstPoint['pointNumber']
                df_for_plot = df.copy()
                df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
                df_for_plot = df_for_plot.reset_index().drop('index',axis=1)

                try:
                    molecule_name = df_for_plot.loc[point_number,'NAME']
                except KeyError:
                    return None
    
                return molecule_name


@app.callback(
    dash.dependencies.Output('chem_name', 'href'),
    [dash.dependencies.Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def return_href(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    datasheet_link = row['PAGE'].iloc[0]
    return datasheet_link


@app.callback(
    Output('chem_img', 'src'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def display_image(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    img_src = row['IMG_URL'].iloc[0]
    return img_src

@app.callback(
    Output('star_rating', 'children'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def display_star(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    star = row['stars'].iloc[0]
    return star

In [ ]:
external_css = ["https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "//fonts.googleapis.com/css?family=Raleway:400,300,600",
                "//fonts.googleapis.com/css?family=Dosis:Medium",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/0e463810ed36927caf20372b6411690692f94819/dash-drug-discovery-demo-stylesheet.css"]


for css in external_css:
    app.css.append_css({"external_url": css})


if __name__ == '__main__':
    app.run_server()


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2018 00:22:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -


[]


127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [10/May/2018 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2018 00:23:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.